#CNN on CIFR Assignment:


1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use Dense Layers (also called fully connected layers), or DropOut.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [ ]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf

In [ ]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 10
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [ ]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
X_train.shape

(50000, 32, 32, 3)

In [ ]:
X_test.shape

(10000, 32, 32, 3)

In [ ]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.1):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (4,4), use_bias=False , kernel_initializer='glorot_uniform',padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.1):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False, kernel_initializer='glorot_uniform' ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [ ]:
num_filter = 20
dropout_rate = 0
l = 12
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(64, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, 32, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, 25, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [ ]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 64)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
print(len(model.layers))

211


In [ ]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam() ,metrics=['accuracy'])

In [ ]:
# adding 
import numpy as np
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
#z-score
mean = np.mean(X_train,axis=(0,1,2,3))
std = np.std(X_train,axis=(0,1,2,3))
X_train = (X_train-mean)/(std+1e-7)
X_test = (X_test-mean)/(std+1e-7)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(X_train)

In [ ]:
#adding 
import os
import datetime
#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',patience=3, factor = 0.8) 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,patience=1)
es = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', mode='max')
import warnings
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint # from Transfer Learning assignment

checkpoint_path = "/gdrive/My Drive/cifar/logs/my_model_weights.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='/gdrive/My Drive/cifar/logs/my_model_weights{epoch}.hdf5',  verbose=1, save_weights_only=False,save_best_only=False, mode='auto') 
log_dir="/gdrive/My Drive/cifar/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

In [ ]:
!mkdir -p saved_model

In [ ]:
model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=100,
                    verbose=1, 
                    validation_data=(X_test, y_test), callbacks= [reduce_lr,tensorboard_callback])

model.save('/gdrive/My Drive/cifar/my_model') 

In [ ]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 2s 7ms/step - loss: 1.0695 - accuracy: 0.6558
Test loss: 1.0694855451583862
Test accuracy: 0.6557999849319458


In [ ]:
def scheduler(epoch, lr):
  lr = 0.0005
  if epoch > 25  :
    lr = 0.0006
  return lr

In [ ]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
model.fit_generator(datagen.flow(X_train,y_train,batch_size=64), steps_per_epoch=300,
                    epochs=100, verbose=1, validation_data=(X_test, y_test), callbacks= [reduce_lr,tensorboard_callback, callback])
model.save('/gdrive/My Drive/cifar/my_model.h5') 

Epoch 1/100
300/300 [==============================] - 48s 159ms/step - loss: 1.3601 - accuracy: 0.5082 - val_loss: 1.3876 - val_accuracy: 0.5106
Epoch 2/100
300/300 [==============================] - 47s 157ms/step - loss: 1.2318 - accuracy: 0.5534 - val_loss: 1.3018 - val_accuracy: 0.5366
Epoch 3/100
300/300 [==============================] - 47s 158ms/step - loss: 1.1433 - accuracy: 0.5908 - val_loss: 1.1205 - val_accuracy: 0.6068
Epoch 4/100
300/300 [==============================] - 47s 158ms/step - loss: 1.0528 - accuracy: 0.6212 - val_loss: 1.4086 - val_accuracy: 0.5560
Epoch 5/100
300/300 [==============================] - 47s 157ms/step - loss: 1.0133 - accuracy: 0.6431 - val_loss: 1.3797 - val_accuracy: 0.5768
Epoch 6/100
300/300 [==============================] - 47s 158ms/step - loss: 0.9710 - accuracy: 0.6527 - val_loss: 1.2345 - val_accuracy: 0.5951
Epoch 7/100
300/300 [==============================] - 47s 158ms/step - loss: 0.9309 - accuracy: 0.6683 - val_loss: 1.3296 -

In [ ]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
model.fit_generator(datagen.flow(X_train,y_train,batch_size=128), steps_per_epoch=300,
                    epochs=100, verbose=1, validation_data=(X_test, y_test), callbacks= [reduce_lr,tensorboard_callback, callback])
model.save('/gdrive/My Drive/cifar/my_model.h5') 

Epoch 1/100
300/300 [==============================] - 77s 256ms/step - loss: 0.5547 - accuracy: 0.8054 - val_loss: 0.7250 - val_accuracy: 0.7793
Epoch 2/100
300/300 [==============================] - 76s 252ms/step - loss: 0.5310 - accuracy: 0.8160 - val_loss: 1.1765 - val_accuracy: 0.6913
Epoch 3/100
300/300 [==============================] - 76s 252ms/step - loss: 0.5086 - accuracy: 0.8227 - val_loss: 0.7394 - val_accuracy: 0.7582
Epoch 4/100
300/300 [==============================] - 76s 252ms/step - loss: 0.4863 - accuracy: 0.8297 - val_loss: 0.6180 - val_accuracy: 0.7951
Epoch 5/100
300/300 [==============================] - 76s 252ms/step - loss: 0.4761 - accuracy: 0.8352 - val_loss: 0.7651 - val_accuracy: 0.7632
Epoch 6/100
300/300 [==============================] - 75s 252ms/step - loss: 0.4634 - accuracy: 0.8372 - val_loss: 0.7212 - val_accuracy: 0.7785
Epoch 7/100
300/300 [==============================] - 75s 252ms/step - loss: 0.4392 - accuracy: 0.8484 - val_loss: 0.5424 -

#Increasing speed GPU

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Found GPU at: /device:GPU:0
Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.395748314000002
GPU (s):
0.04878333900001053
GPU speedup over CPU: 69x


In [ ]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 8s 25ms/step - loss: 0.5384 - accuracy: 0.8439
Test loss: 0.5384398698806763
Test accuracy: 0.8439000248908997


## **Conclusion:**

Test Accuracy: 84.39 \\
Validation Accuracy: 88.74 \\
Epochs: 200 \\
Train Accuracy: 96.07 \\
